In [1]:
import numpy as np
import dns.message, dns.query, dns.rdataclass, dns.rdatatype, dns.flags, dns.exception, dns.name
import pandas as pd

In [2]:
%%bash
ls -lh *.pickle

-rw-rw-r-- 1 nils nils 897M Sep  7 19:20 tranco-1000000-data.pickle
-rw-rw-r-- 1 nils nils  13M Sep  7 14:07 tranco-10000-data.pickle
-rw-rw-r-- 1 nils nils 163K Sep  7 11:18 tranco-1000-data.pickle
-rw-rw-r-- 1 nils nils  20K Sep  6 11:39 tranco-100-data.pickle


In [3]:
data = pd.read_pickle(f'tranco-1000000-data.pickle')

In [4]:
data['bootstrapable_immediate'] = data['signed'] & data['bootstrapable']

data['ns_mnames'] = data.apply(lambda row: {soa[0].mname.to_text() for soa in row['ns_soas'] if soa}, axis=1)
data['ns_mnames_num'] = data.apply(lambda row: len(row['ns_mnames']), axis=1)
data['ns_mname'] = data.apply(lambda row: None if row['ns_mnames_num'] != 1 else next(iter(row['ns_mnames'])), axis=1)

data['ns_rnames'] = data.apply(lambda row: {soa[0].rname.to_text() for soa in row['ns_soas'] if soa}, axis=1)
data['ns_rnames_num'] = data.apply(lambda row: len(row['ns_rnames']), axis=1)
data['ns_rname'] = data.apply(lambda row: None if row['ns_rnames_num'] != 1 else next(iter(row['ns_rnames'])), axis=1)

In [5]:
print(f"Proportion of failed queries................................: {1 - len(data) / 10**6:7.2%}")
print(f"Sample size.................................................: {len(data):7}")
print()
print(f"Total proportion of SECURE zones............................: {data['secure'].mean():7.2%}")
print(f"Total proportion of SIGNED zones............................: {data['signed'].mean():7.2%}")
print()
print(f"Total proportion of zones with ALL  name server names SECURE: {np.floor(data['ns_secure']).mean():7.2%}")
print(f"Total proportion of zones with ALL  name server names SIGNED: {np.floor(data['ns_signed']).mean():7.2%}")
print(f"Total proportion of zones with SOME name server names SECURE: {np.ceil(data['ns_secure']).mean():7.2%}")
print(f"Total proportion of zones with SOME name server names SIGNED: {np.ceil(data['ns_signed']).mean():7.2%}")
print()
print(f"Total proportion of      bootstrapable zones................: {data['bootstrapable'].mean():7.2%}")
print(f"Total proportion of weak bootstrapable zones................: {data['bootstrapable_weak'].mean():7.2%}")

Proportion of failed queries................................:   3.80%
Sample size.................................................:  962012

Total proportion of SECURE zones............................:   4.47%
Total proportion of SIGNED zones............................:   5.87%

Total proportion of zones with ALL  name server names SECURE:  24.14%
Total proportion of zones with ALL  name server names SIGNED:  24.55%
Total proportion of zones with SOME name server names SECURE:  25.36%
Total proportion of zones with SOME name server names SIGNED:  26.14%

Total proportion of      bootstrapable zones................:  21.77%
Total proportion of weak bootstrapable zones................:  22.66%


In [6]:
data.groupby(['tld']).agg({
    'domain': ['count'],
    'bootstrapable': ['mean', 'sum'],
    'bootstrapable_immediate': ['mean', 'sum'],
    'signed': ['mean'],
    'secure': ['mean'],
    'ns_secure_all': ['mean'],
}).sort_values(('bootstrapable_immediate', 'sum'), ascending=False).head(10)

domain bootstrapable         bootstrapable_immediate          signed  \
      count          mean     sum                    mean   sum      mean   
tld                                                                         
com  493152      0.235917  116343                0.008354  4120  0.039237   
org   68720      0.180384   12396                0.007625   524  0.043001   
net   43894      0.236274   10371                0.009796   430  0.051100   
nl    10329      0.162262    1676                0.020331   210  0.530351   
io     7134      0.343706    2452                0.017662   126  0.084104   
uk    20102      0.188936    3798                0.005870   118  0.027360   
co     7089      0.302723    2146                0.015940   113  0.078713   
ru    31435      0.137649    4327                0.003467   109  0.058152   
fr     8550      0.120819    1033                0.012515   107  0.120000   
in     9208      0.287250    2645                0.010643    98  0.058102   

       secure ns_secure_all  
         mean          mean  
tld                          
com  0.028111      0.248692  
org  0.031534      0.194674  
net  0.037386      0.253246  
nl   0.497822      0.510795  
io   0.060976      0.379731  
uk   0.018854      0.197095  
co   0.059529      0.354352  
ru   0.004740      0.139590  
fr   0.098363      0.129942  
in   0.042246      0.324826

In [7]:
data.groupby(['tld', 'ns_mname']).agg({
    'domain': ['count'],
    'bootstrapable': ['mean', 'sum'],
    'bootstrapable_immediate': ['mean', 'sum'],
    'signed': ['mean'],
    'secure': ['mean'],
    'ns_secure_all': ['mean'],
}).sort_values(('bootstrapable_immediate', 'sum'), ascending=False).head(10)

domain bootstrapable         bootstrapable_immediate  \
                          count          mean     sum                    mean   
tld ns_mname                                                                    
com ns3.cloudflare.com.  110506      0.961984  106305                0.032967   
org ns3.cloudflare.com.   11786      0.949262   11188                0.036484   
net ns3.cloudflare.com.    9973      0.956884    9543                0.036398   
com ns1.loopia.se.          158      1.000000     158                0.987342   
io  ns3.cloudflare.com.    2536      0.910883    2310                0.049290   
co  ns3.cloudflare.com.    2107      0.966777    2037                0.052682   
ru  ns3.cloudflare.com.    4255      0.992479    4223                0.024207   
in  ns3.cloudflare.com.    1790      0.978771    1752                0.054190   
uk  ns3.cloudflare.com.    3375      0.982519    3316                0.028444   
br  ns3.cloudflare.com.    1433      0.931612    1335                0.059316   

                                 signed    secure ns_secure_all  
                          sum      mean      mean          mean  
tld ns_mname                                                     
com ns3.cloudflare.com.  3643  0.070983  0.038016           1.0  
org ns3.cloudflare.com.   430  0.087222  0.050738           1.0  
net ns3.cloudflare.com.   363  0.079515  0.043116           1.0  
com ns1.loopia.se.        156  0.987342  0.000000           1.0  
io  ns3.cloudflare.com.   125  0.138407  0.089117           1.0  
co  ns3.cloudflare.com.   111  0.085904  0.033223           1.0  
ru  ns3.cloudflare.com.   103  0.031727  0.007521           1.0  
in  ns3.cloudflare.com.    97  0.075419  0.021229           1.0  
uk  ns3.cloudflare.com.    96  0.045926  0.017481           1.0  
br  ns3.cloudflare.com.    85  0.127704  0.068388           1.0

In [8]:
data.groupby(['ns_mname', 'bootstrapable'], dropna=False).agg({'domain': ['count']}).sort_values(('domain', 'count'), ascending=False).head(10)

,,domain
,,count
ns_mname,bootstrapable,
ns3.cloudflare.com.,True,186524
NaN,False,136328
ans01.domaincontrol.com.,False,74874
heidi.ns.cloudflare.com.,False,28797
ns1.hichina.com.,False,16698
edns.registrar-servers.com.,False,16019
ns5.googledomains.com.,False,10979
m1.dns.com.,False,9393


In [9]:
data.groupby(['ns_rname', 'bootstrapable'], dropna=False).agg({'domain': ['count']}).sort_values(('domain', 'count'), ascending=False).head(10)

,,domain
,,count
ns_rname,bootstrapable,
dns.cloudflare.com.,True,188746
awsdns-hostmaster.amazon.com.,False,82100
dns.jomax.net.,False,76222
dns.cloudflare.com.,False,58400
NaN,False,52354
hostmaster.nsone.net.,False,17334
hostmaster.hichina.com.,False,17017
support.namecheap.com.,False,11723


In [10]:
data[data['bootstrapable'] & data['ns_mname']].groupby(['tld', 'ns_mname']).agg({'domain': ['count']}).reset_index().sort_values(('domain', 'count'), ascending=False).head(10)

,tld,ns_mname,domain
,,,count
866,com,ns3.cloudflare.com.,106305
2581,org,ns3.cloudflare.com.,11188
2130,net,ns3.cloudflare.com.,9543
2832,ru,ns3.cloudflare.com.,4223
3204,uk,ns3.cloudflare.com.,3316
1699,io,ns3.cloudflare.com.,2310
714,com,ns1.dns-parking.com.,2244
445,co,ns3.cloudflare.com.,2037
1611,in,ns3.cloudflare.com.,1752


In [11]:
data[data['ns_rname'].isna() & data['bootstrapable']][['zone', 'ns_rnames']].head(10)

,zone,ns_rnames
778,bigcartel.com.,"{hostmaster.nsone.net., dns.cloudflare.com.}"
930,vtv.vn.,"{hostmaster.dotvndns.vn., hostmaster.dotvndns...."
1504,cbs.com.,"{hostmaster.nsone.net., dns.cloudflare.com.}"
1936,cc.com.,"{hostmaster.nsone.net., dns.cloudflare.com.}"
2320,mtv.com.,"{hostmaster.nsone.net., dns.cloudflare.com.}"
2461,last.fm.,"{hostmaster.nsone.net., dns.cloudflare.com.}"
2540,cbssports.com.,"{hostmaster.nsone.net., dns.cloudflare.com.}"
2683,cbsnews.com.,"{hostmaster.nsone.net., dns.cloudflare.com.}"
3290,dickssportinggoods.com.,"{hostmaster.cscdns.net., premiumdns.support.ne..."
4472,simonandschuster.com.,"{hostmaster.nsone.net., dns.cloudflare.com.}"


In [12]:
data.iloc[930]['ns_rnames']

{'hostmaster.dotvndns.com.', 'hostmaster.dotvndns.vn.'}